# FINC621 Winter 2018-19 Lab Worksheet 03

## subtitle: Variance, Covariance, Correlation & Causality (finc621-lab03)

#### author: "Yu Jia"

#### date: "11/28/2018"

#### output:
  #### html_notebook: default
  #### html_document: default
  

### About

In this worksheet we look at different variance, covariance, volatility, and causality calculations. We finish with a short matematical proof (no R required).  

### Setup

Remember to always set your working directory to the source file location. Go to 'Session', scroll down to 'Set Working Directory', and click 'To Source File Location'. Read carefully the below and follow the instructions to complete the tasks and answer any questions.  Submit your work to RPubs as detailed in previous notes. 

### Note

For clarity, tasks/questions to be completed/answered are highlighted in red color (color visible only in preview mode) and numbered according to their particular placement in the task section.  Type your answers outside the red color tags!


Quite often you will need to add your own code chunk. Execute sequentially all code chunks, preview, publish, and submit link on Sakai following the naming convention. Make sure to add comments to your code where appropriate. Use own language!


**Any sign of plagiarism, will result in dissmissal of work!**

### Task 1: Variance, Covariance, and Volatility

This task follows the two examples in the book `R Example 2.5/p. 58` and `R Example 2.6/p. 66

<font color=red>##### 1A) Calculate the correlation and covariance matrix of the adjusted daily log returns for four different stocks of your choice. Explain your observations in terms of potential relationships.</font>

In [1]:
import pandas as pd
import numpy as np
import pandas_datareader.data as web

In [2]:
symbol=['TSLA','BABA','AMZN','FB']
df=web.DataReader(symbol,'yahoo',2015-1-1,2019-1-1)
# tsla daily log return
Treturn=np.log(df['Adj Close','TSLA']).diff().dropna()
# pd.to_numeric(Treturn,downcast='float')
# baba daily log return
Breturn=np.log(df['Adj Close','BABA']).diff().dropna()
# amzn daily log return
Areturn=np.log(df['Adj Close','AMZN']).diff().dropna()
# fb daily log return
Freturn=np.log(df['Adj Close','FB']).diff().dropna()
# create matrix 
M=pd.concat([Treturn,Breturn,Areturn,Freturn],axis=1)


In [3]:
# covariance of matrix
M.cov()

Adj Close                              
                    TSLA      BABA      AMZN        FB
Adj Close TSLA  0.000967  0.000148  0.000173  0.000191
          BABA  0.000148  0.000426  0.000134  0.000110
          AMZN  0.000173  0.000134  0.000373  0.000176
          FB    0.000191  0.000110  0.000176  0.000458

In [4]:
# correlation of matrix
M.corr()

Adj Close                              
                    TSLA      BABA      AMZN        FB
Adj Close TSLA  1.000000  0.288648  0.288014  0.286310
          BABA  0.288648  1.000000  0.331706  0.313776
          AMZN  0.288014  0.331706  1.000000  0.425165
          FB    0.286310  0.313776  0.425165  1.000000

<font color=red>##### 1B) Calculate the three types of volatility for a particular stock of your choice. Consider a time window extending one year back from most recent obtainable closing day price. Order the three estimates from low to high volatility and explain how the ordering makes sense.</font>

In [5]:
# gain TSLA adjusted Close price data from 2017-9 to 2018-10
# tsla=df.loc[['2017-9','2018-10'],['Adj Close','TSLA']]


In [6]:
import math

def calculate_parkinson(item):
    """ 
    Calculates the Parkinson volatility of a MultiIndexed Pandas DataFrame 
  
    Parameters: 
    item (DataFrame)
  
    Returns: 
    float: The calculated volatility
    """
    
    # First we need to know how many items are in the data set
    m = int(item['High'].count())
    
    # We set the sum to zero and then iterate through the data
    sum = 0.0
    for index, row in item.iterrows():
        high = float(row['High'])
        low = float(row['Low'])
        
        if math.isnan(low): # Check to see if there is null data in the denominator
            m = m - 1 # If so, decrease the number of items in the data set, and move to the next iteration
            continue
            
        sum += (math.log(high / low)) ** 2
    
    # We perform the rest of the calcuation and then send result value back
    return(math.sqrt((1 / (4 * math.log(2))) * (1 / m) * sum))
   
def calculate_garman_klass(item):
    """ 
    Calculates the Garman Klass volatility of a MultiIndexed Pandas DataFrame 
  
    Parameters: 
    item (DataFrame)
  
    Returns: 
    float: The calculated volatility
    """
    
    # First we need to know how many items are in the data set
    m = int(item['High'].count())
    
    # We set the sum to zero and then iterate through the data
    sum = 0.0
    for index, row in item.iterrows():
        high = float(row['High'])
        low = float(row['Low'])
        closing = float(row['Close'])
        opening = float(row['Open'])

        if math.isnan(low) or math.isnan(opening): # Check to see if there is null data in the denominator
            m = m - 1 # If so, decrease the number of items in the data set, and move to the next iteration
            continue
            
        sum += (0.5 * (math.log(high / low)) ** 2 - (2 * math.log(2) - 1) * (math.log(closing / opening) ** 2))
    
    # We perform the rest of the calcuation and then send result value back
    return(math.sqrt(1 / m * sum))

In [7]:
# Isolate the data from the data frame
tesla = df.loc[:,(df.columns.get_level_values(1) == 'TSLA')]
alibaba = df.loc[:,(df.columns.get_level_values(1) == 'BABA')]
amazon = df.loc[:,(df.columns.get_level_values(1) == 'AMZN')]
facebook = df.loc[:,(df.columns.get_level_values(1) == 'FB')]

# Set up a dictionary to hold the values
p_volatility = {}
p_volatility["Tesla"] = calculate_parkinson(tesla)
p_volatility["Alibaba"] = calculate_parkinson(alibaba)
p_volatility["Amazon"] = calculate_parkinson(amazon)
p_volatility["Facebook"] = calculate_parkinson(facebook)

# Set up a dictionary to hold the values
gk_volatility = {}
gk_volatility["Tesla"] = calculate_garman_klass(tesla)
gk_volatility["Alibaba"] = calculate_garman_klass(alibaba)
gk_volatility["Amazon"] = calculate_garman_klass(amazon)
gk_volatility["Facebook"] = calculate_garman_klass(facebook)

# You can print all the values like this:
for item in p_volatility:
    print("Parkinson volatility for ", item, "is: ", p_volatility[item])
    
for item in gk_volatility:
    print("Garman and Klass volatility for ", item, "is: ", gk_volatility[item])

# You can also print the values 1 at a time like this:
# print(p_volatility["Tesla"])

Parkinson volatility for  Tesla is:  0.026187376988483845
Parkinson volatility for  Alibaba is:  0.035604425898633905
Parkinson volatility for  Amazon is:  0.014364686178111132
Parkinson volatility for  Facebook is:  0.016842010608405292
Garman and Klass volatility for  Tesla is:  0.026340640417009974
Garman and Klass volatility for  Alibaba is:  0.035529724976171795
Garman and Klass volatility for  Amazon is:  0.014384841148138651
Garman and Klass volatility for  Facebook is:  0.01683996237813042


In [ ]:
import sys
!{sys.executable} -m 

$ git clonen http://github.com/volatilityfoundation/volatility.git

In [ ]:
# This will create a volatility folder that contains the source code and you can run volatility derectory from there.
import sys
! git clone http://github.com/volatilityfoundation/volatility.git

### Task 2: Auto-Correlation and Auto-Regression

Follow the example in the book  `R Example 3.2/p. 74` and `R Example 4.1/p. 115`

<font color=red>##### 2A) Calculate the ACF for a stock of your choice. Consider both the log return and squared log return. Interpret your results in terms of possible existence of autocorrelation.</font>

In [ ]:
import sys
!{sys.executable} -m conda install statsmodels.graphics.tsaplots

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
from matplotlib import pyplot
import matplotlib.pyplot as plt

In [ ]:
# calculate daily log return of TSLA and squared log return of TSLA
TSLA_logreturn=np.log(df['Adj Close','TSLA']).diff().dropna()*100
TSLA_Slogreturn=np.log(df['Adj Close','TSLA']).diff().dropna()**2*100

In [ ]:
# # plot autocorrelation of daily log return of TSLA
fig=plot_acf(TSLA_logreturn,lags=30)
fig.set_size_inches(15.5,10.5,forward=True)

In [ ]:
# plot autocorrelation of square daily log return of TSLA
figs=plot_acf(TSLA_Slogreturn,lags=30)
figs.set_size_inches(15.5,10.5,forward=True)

<font color=red>##### 2B) Plot the exchange rate for USD versus another currency of your choice. Interpret your results in terms of behavior.</font>

In [ ]:
# gain U.S/Euro data from FRED
euro=web.DataReader('DEXUSEU','fred')
euro.plot(figsize=(12,9))

<font color=red>##### 2C) Test for the possible existence of an underlying AR(1) – Markov process in your exchange rate currency pair. To this end, plot the ACF and the partial ACF (PACF). Interpret your results.  Clearly refer to the lags, and their impacts in determining the order.</font>

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf

In [ ]:
# plot partial autocorrelation of exchange rate of currency
fig_3=plot_pacf(euro, lags=15)
fig_3.set_size_inches(15.5,10.5,forward=True)

### Task 3: Granger Causality Test

To conduct this test the package `lmtest` will be required, as already done in the code chunk below.

<font color=red>##### 3A) Include below the code chunk to solve for 3.5.7 R Lab/p. 106.  Write your conclusions.</font>

In [ ]:
package_contents('statsmodels')

In [ ]:
import sys
!{sys.executable} -m pip install statsmodels

In [ ]:
from statsmodels.tsa.stattools import grangercausalitytests

In [ ]:
USMoney=pd.read_excel("/Users/hangxigudeaoqi/Desktop/USMoney.xlsx")
USMoney.head()

In [ ]:
a=USMoney['gnp']
b=USMoney['m1']
x=np.column_stack((a,b))

In [ ]:
grangercausalitytests(x,maxlag=15,addconst=True,verbose=True)

In [ ]:
# b is m1, a is gnp
y=np.column_stack((b,a))

In [ ]:
grangercausalitytests(y,maxlag=15,addconst=True,verbose=True)

<font color='red'>The Null hypothesis for grangercausality is that time series in the second column doesn't granger cause the time series in the first column. Granger causality means that past values of the second column have a statistically significant effect on the current value of the first column, taking past values of the first column into account as regressors. We reject the null hypothesis that the second column doesn't granger cause the first column if the p-value are below a desiredd size of the test.

<font color=red>##### 3B) Briefly describe the data in terms of time range and variables. Similar to the linear autoregressive model described in class, write the mathematical regression model solved in each Granger test, including the proper order. Use naming conventions, and notations more reflective of the data set considered for  `USMoney`.

$Y_t=a_0+a_1Y_1+a_2Y_2+a_3Y_3+b_1 X_1+b_2X_2+b_3X_3$
In the first Granger causality test, x is the M1 and y is the GNP. 
When p-value is below 0.5, one rejects null hypothesis that M1 doesn't cause the GNP. In other words, the M1 causes the GNP.
In the second Granger causality test, x is the GNP and y is the M1.
When p-value is abover 0.5, one accpect null hypothesis. It means that the GNP doesn't causes the M1.

### Task 4: Mathematical Proof

<font color=red>##### 4A) Prove the two results in Eq (2.32)/p. 53.  No R-coding is needed here.  Clearly show your steps. Hint: Use the definition of $E(X^n)$ for X-log normally distributed.   Observe also that $Var(X) = E(X^2)-E^2(X)$ for any random variable X.<font/>

The moments of the variable X are
$E(X^n)=e^{nu+1/2n^2\sigma^2}$,n>0
assuming ${R_{t}}$ is log-normally distributed, since $r_{t}=ln(R_{t}+1)$
$e^{r}=R+1$, then $R=e^{r}-1$,in $E(X^n)=e^{nu+1/2n^2\sigma^2}$

when n=1: $E(R)=u_R=e^{u_r+\sigma^2_r/2}-1$

when n=2,input in formula , the result $E(x^2)=E(R^2)-E^2(R)=e^{2u_r+2\sigma^2_r}-(e^{u_r+\sigma^2_r/2})^2=e^{2u_r+\sigma^2_r}(e^{\sigma^2_r}-1)$
